In [1]:
import tensorflow as tf
import numpy as np
from numpy import random
random.seed(567)
tf.set_random_seed(777)  # reproducibility
sess = tf.Session()

/Users/stephaniexia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 1
batch_size = 3
num_boxes=5
#display_step = 100

# Network Parameters
input_size = num_boxes*1029
timesteps_size = 30 # timesteps
hidden_nodes1 = 150 # hidden layer num of features
hidden_nodes2 = 300
n_classes = 2
dim_bbox=num_boxes*4
dim_class_logits=num_boxes*n_classes



In [3]:
#num_boxes=5
#from numpy import random
#random.seed(567)
x_data=random.random((3,30,num_boxes*1029)).astype('float32')
print('x_data.shape:',x_data.shape)
print('x[0].shape:',x_data[0].shape)
print('x[0][0].shape:',x_data[0][0].shape)
#X = tf.placeholder("float32", [None, timesteps_size, input_size])
X=x_data
X=tf.Variable(x_data)

# Tensorflow LSTM cell requires 2x n_hidden length (state & cell)
#Y = tf.placeholder("float32", [None, dim_gt])
#Y=y_data

#print('x[0][0][0].shape:',x_data[0][0][0].shape,x_data[0][0][0])
#y_data=random.random((3,num_boxes*7)).astype('float32')
class_ids=random.randint(0,n_classes,size=(batch_size,num_boxes))
print(class_ids)
Y_class_ids=tf.Variable(class_ids)
print(Y_class_ids)
gt_boxes=random.random(([batch_size,num_boxes,4])).astype('float32')
#print(gt_boxes)
Y_gt_boxes=tf.Variable(gt_boxes)

x_data.shape: (3, 30, 5155)
x[0].shape: (30, 5155)
x[0][0].shape: (5155,)
[[1 0 0 1 0]
 [1 1 0 0 1]
 [0 1 0 1 1]]
<tf.Variable 'Variable_1:0' shape=(3, 5) dtype=int64_ref>


In [ ]:
#TODO: pad zeros for detections:
#x_feature:1024
#x_box:4
#x_class:num_classes-1 ????
input_feature=['None*num_obj*1024']
X_det=np.zeros((len(input_feature),num_boxes,1029))
X_det=np.zeros((batch_size,timesteps_size,num_boxes,1029))

from scipy.optimize import linear_sum_assignment
def generate_distance_matrix(pic1,pic2,features1,features2):
    pic1=np.array(pic1)
    pic2=np.array(pic2)
    pic1_feature=features1
    pic2_feature=features2
    #pad 0, keep same number of boxes in two pictures
    for pad_num in range(abs(len(pic1)-len(pic2))):
        if len(pic1)<len(pic2):
            pic1=np.row_stack((pic1,np.zeros(4)))
            pic1_feature=np.row_stack((features1,np.zeros(1024)))
        if len(pic2)<len(pic1):
            pic2= np.row_stack((pic2,np.zeros(4)))
            pic2_feature=np.row_stack((features2,np.zeros(1024)))
    #initialize a distance matrix
    distance_matrix=np.zeros((len(pic1),len(pic2))) #len(pic1)=len(pic2)
    #calculate distance matrix
    for i in range(len(pic1)):
        box_i=pic1[i]
        #find center of the box_i
        cent_i =np.array([(box_i[1] + (box_i[3] - box_i[1]) / 2), (box_i[0] + (box_i[2] - box_i[0]) / 2)])
        for j in range(len(pic2)):
            box_j=pic2[j]
            cent_j = np.array([(box_j[1] + (box_j[3] - box_j[1]) / 2), (box_j[0] + (box_j[2] - box_j[0]) / 2)])
            #calculate Euclidean distance
            distance_matrix[i,j]=np.sqrt(np.sum(np.square(cent_i-cent_j)))
    return distance_matrix,pic1_feature,pic2_feature

def box_pair_assignment(pic1,pic2,features1,features2):
    distance_matrix,pic1_feature,pic2_feature=generate_distance_matrix(pic1, pic2,features1,features2)
    #assign picture into pairs
    pic1_ind,pic2_ind=linear_sum_assignment(distance_matrix)
    #pairs=np.array((pic1_ind,pic2_ind))
    return pic1_ind,pic2_ind,pic1_feature,pic2_feature

graph:

In [4]:
#print(h_out)
def fc_bbox(input):
    w1 = tf.get_variable('weight1', shape=[hidden_nodes2, dim_bbox], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable('bias1', [dim_bbox], initializer=tf.zeros_initializer())
    h_fc_bbox = tf.nn.relu(tf.nn.xw_plus_b(input, w1, b1), name='relu')
    print('fc_bbox completed')
    return h_fc_bbox

def fc_class_logits(input):
    w1 = tf.get_variable('weight1', shape=[hidden_nodes2, dim_class_logits], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable('bias1', [dim_class_logits], initializer=tf.zeros_initializer())
    h_fc_logits = tf.nn.relu(tf.nn.xw_plus_b(input, w1, b1), name='relu')
    print('fc_class_logits completed')
    return h_fc_logits

def fc_2(input):
    w2 = tf.get_variable('weight2', shape=[hidden_nodes2,50], initializer=tf.truncated_normal_initializer())
    b2 = tf.get_variable('bias2', [50], initializer=tf.zeros_initializer())
    h_fc_2 = tf.nn.relu(tf.nn.xw_plus_b(input, w2, b2), name='relu')
    print('fc_2 completed')
    return h_fc_2



In [5]:
#class_ids=random.randint(0,n_classes,size=(batch_size,num_boxes))
#print(class_ids)
#Y_class_ids=tf.Variable(class_ids)
#print(Y_class_ids)
def class_loss_graph(h_fc_logits,Y_class_ids):
    print('start computing class_loss_graph')
    h_fc_logits=tf.reshape(h_fc_logits[:,-1,:,:],(batch_size,num_boxes,n_classes))
    class_loss= tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y_class_ids, logits=h_fc_logits)
    #print(sess.run(class_loss))
    return class_loss
#sess=tf.Session()
#sess.run(tf.global_variables_initializer())
#class_loss=tf.reduce_mean(class_loss_graph(h_fc_logits,Y_class_ids))
#print(sess.run(class_loss))

In [6]:

#gt_boxes=random.random(([batch_size,num_boxes,4])).astype('float32')
#print(gt_boxes)
#Y_gt_boxes=tf.Variable(gt_boxes)
def smooth_l1_loss(y_true, y_pred):
    diff = tf.math.abs(y_true - y_pred)
    less_than_one = tf.cast(tf.less(diff, 1.0), "float32")
    loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
    return loss
def box_loss_graph(h_fc_logits,Y_class_ids,h_fc_bbox,Y_gt_boxes):
    print('box_loss_graph')
    logits=tf.reshape(h_fc_logits[:,-1,:,:],(batch_size,num_boxes,n_classes))
    correct_prediction=tf.cast(tf.equal(tf.argmax(logits,2),Y_class_ids),tf.float32)
    #print(sess.run(correct_prediction))
    pred_bbox=tf.reshape(h_fc_bbox[:,-1,:,:],(batch_size,num_boxes,4))
    #print(sess.run(pred_bbox))
    _box_loss=smooth_l1_loss(y_true=Y_gt_boxes, y_pred=pred_bbox)
    #print(sess.run(_box_loss))
    correct_box_loss=tf.math.multiply(tf.reshape(correct_prediction,(3,5,1)),_box_loss)
    #print(sess.run(correct_box_loss))
    box_loss=tf.reduce_mean(correct_box_loss)
    #print(sess.run(box_loss))
    #print(sess.run(tf.argmax(logits,2)))
    return box_loss
#sess=tf.Session()
#sess.run(tf.global_variables_initializer())
#box_loss=box_loss_graph(h_fc_logits,Y_class_ids,h_fc_bbox,Y_gt_boxes)

    
    

In [7]:
#sess=tf.Session()
#sess.run(tf.global_variables_initializer())
def smooth_loss_graph(h_fc_bbox):
    print('smooth_loss_graph')
    smooth_loss=0
    for i in range(batch_size):
        #_h_fc_1=h_fc_1[i]
        for j in range(timesteps_size -1):
            #print('smooth_loss_graph:{}/{}'.format(i,j))
            smooth_loss+=tf.reduce_mean(smooth_l1_loss(h_fc_bbox[i][j],h_fc_bbox[i][j+1]))
            #print(sess.run(smooth_loss))
    smooth_loss=smooth_loss/(batch_size*(timesteps_size -1))
    return smooth_loss
#print(sess.run(smooth_loss))

In [8]:
def association_loss(h_fc_2):
    print('association_loss')
    asso_loss=0
    for i in range(batch_size-1):
            _asso_loss=tf.reduce_sum(tf.math.multiply(h_fc_2[i],h_fc_2[i+1]),-1)
            #print(sess.run(_asso_loss))
            asso_loss+=tf.reduce_mean(_asso_loss)
    asso_loss=asso_loss/(batch_size-1)
    return asso_loss
#sess=tf.Session()
#sess.run(tf.global_variables_initializer())
#asso_loss=association_loss(h_fc_2)
#sess.run(asso_loss)

In [9]:
print('start lstm')
lstm_cell_1 = tf.nn.rnn_cell.LSTMCell(hidden_nodes1, state_is_tuple=True)
lstm_cell_2 = tf.nn.rnn_cell.LSTMCell(hidden_nodes2, state_is_tuple=True)
mul_lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell_1,lstm_cell_2], state_is_tuple = True)
init_state = mul_lstm_cell.zero_state(batch_size, dtype=tf.float32)
outputs, state = tf.nn.dynamic_rnn(mul_lstm_cell, inputs=X, initial_state=init_state, time_major=False)
h_out = outputs
h_state = state[-1][-1]
#print(h_out.shape)
#print(tf.reshape(h_state[0],(1,-1)))
#sess.run(tf.global_variables_initializer())
#print(sess.run(h_state))

h_fc_bbox=[]
with tf.variable_scope('fc_bbox') as scope:
    for i in range(batch_size):
        try:            
            #var=tf.get_variable('fc_1/weight1')
            _h_fc_bbox = fc_bbox(h_out[i])
            #tf.stack([h_fc_1,_h_fc_1 ],axis=0)
        except ValueError:
            scope.reuse_variables()
            _h_fc_bbox = fc_bbox(h_out[i])
        h_fc_bbox.append(_h_fc_bbox)
#print(h_fc_bbox)
h_fc_bbox=tf.stack(h_fc_bbox,axis=0)
#print(h_fc_bbox)
h_fc_bbox=tf.reshape(h_fc_bbox,(3,30,5,-1))
#print(h_fc_bbox)
#print(h_fc_1)

h_fc_logits=[]
with tf.variable_scope('fc_logits') as scope:
    for i in range(batch_size):
        try:            
            #var=tf.get_variable('fc_1/weight1')
            _h_fc_logits = fc_class_logits(h_out[i])
            #tf.stack([h_fc_1,_h_fc_1 ],axis=0)
        except ValueError:
            scope.reuse_variables()
            _h_fc_logits = fc_class_logits(h_out[i])
            #h_fc_1=tf.stack([h_fc_1,_h_fc_1 ],axis=0)
        h_fc_logits.append(_h_fc_logits)
        #print(h_out[i])
        #_output=fc_1(h_out[i])
        #_h_fc_1 = tf.nn.relu(tf.nn.xw_plus_b(h_out[i], w1, b1), name='relu')
#print(h_fc_logits)
h_fc_logits=tf.stack(h_fc_logits,axis=0)
#print(h_fc_logits)
h_fc_logits=tf.reshape(h_fc_logits,(3,30,5,-1))
#print(h_fc_logits)

h_fc_2=[]
with tf.variable_scope('fc_2') as scope:
    for i in range(batch_size):
        try:            
            _h_fc_2 = fc_2(tf.reshape(h_state[i],(1,-1)))
            #tf.stack([h_fc_2,_h_fc_2 ],axis=-1)
        except ValueError:
            scope.reuse_variables()
            _h_fc_2 = fc_2(tf.reshape(h_state[i],(1,-1)))
            #print(_h_fc_2)
        h_fc_2.append(_h_fc_2 )
            #print(h_fc_2)
#sess.run(tf.global_variables_initializer())
#print(sess.run(h_fc_2[0]))
h_fc_2=tf.stack(h_fc_2,axis=0)
#print(h_fc_2)
#h_fc_2=tf.reshape(h_fc_2,(3,5,-1))
#print(h_fc_2)

class_loss=tf.reduce_mean(class_loss_graph(h_fc_logits,Y_class_ids))
box_loss=box_loss_graph(h_fc_logits,Y_class_ids,h_fc_bbox,Y_gt_boxes)
smooth_loss=smooth_loss_graph(h_fc_bbox)
asso_loss=association_loss(h_fc_2)
total_loss=class_loss + box_loss +smooth_loss+ asso_loss
#print(sess.run(total_loss))
train = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

#correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(y_input,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())
is_train=True
save_path='/Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial/'
saver=tf.train.Saver(max_to_keep=1)
if is_train:
    max_acc=0
    f=open(save_path+'acc.text','w')
    for i in range(10):
        cost, _ = sess.run([total_loss, train])
        cls=sess.run(class_loss)
        box=sess.run(box_loss)
        asso=sess.run(asso_loss)
        smooth=sess.run(smooth_loss)
        print("train cost={:.6f},class cost={:.6f},box cost={:.6f},asso cost={:.6f},smooth cost={:.6f}".format(cost, cls, box,asso,smooth))
    saver.save(sess,save_path+'weights.ckpt',global_step=i)

start lstm
fc_bbox completed
fc_bbox completed
fc_bbox completed
fc_class_logits completed
fc_class_logits completed
fc_class_logits completed
fc_2 completed
fc_2 completed
fc_2 completed
start computing class_loss_graph
box_loss_graph
smooth_loss_graph
association_loss
train cost=108.487427,class cost=1.550079,box cost=0.516131,asso cost=200.914612,smooth cost=0.013598
train cost=202.994415,class cost=1.922388,box cost=0.368404,asso cost=23.215086,smooth cost=0.013607
train cost=25.519485,class cost=1.709337,box cost=0.455378,asso cost=4.128832,smooth cost=0.013136
train cost=6.306683,class cost=1.494356,box cost=0.480459,asso cost=0.226131,smooth cost=0.012988
train cost=2.213934,class cost=1.341630,box cost=0.504527,asso cost=0.067405,smooth cost=0.013206
train cost=1.926767,class cost=1.208666,box cost=0.525792,asso cost=0.091544,smooth cost=0.013687
train cost=1.839689,class cost=1.088366,box cost=0.540962,asso cost=0.000000,smooth cost=0.014298
train cost=1.643626,class cost=0.97

def smooth_l1_loss(y_true, y_pred):
    diff = tf.math.abs(y_true - y_pred)
    less_than_one = tf.cast(tf.less(diff, 1.0), "float32")
    loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
    return loss
def smooth_loss(h_fc_bbox):
    smooth_loss=0
    for i in range(batch_size):
        #_h_fc_bbox=h_fc_bbox[i]
        for j in range(timesteps_size -1):
            for bbox in range(num_boxes):
                bbox_1=h_fc_bbox[i][j][box_num*4:(box_num+1)*4]
                print(bbox_1)
                bbox_2=h_fc_bbox[i][j+1][box_num*4:(box_num+1)*4]
                print(bbox_2)
                smooth_loss+=tf.reduce_mean(tf.square(_h_fc_1[j][i]-_h_fc_1[j+1][i]))

            
    loss = K.switch(tf.size(target_bbox) > 0,
                    smooth_l1_loss(y_true=target_bbox, y_pred=pred_bbox),
                    tf.constant(0.0))
    loss = K.mean(loss)
    loss = K.reshape(loss, [1, 1])
    return loss

    return smooth_loss
def prediciton_loss(h_fc_1,Y):
    prediction_loss=0
    for i in range(batch_size):
        #print(Y[i])
        #print(sess.run(h_fc_1[i][-1]))
        cross_entropy = tf.reduce_mean(Y[i] *h_fc_1[i][-1])
        #print(sess.run(cross_entropy))
        prediction_loss+=cross_entropy
    return prediction_loss
def association_loss(h_fc_2):
    asso_loss=0
    for i in range(batch_size-1):
            _asso_loss=tf.reduce_mean(h_fc_2[i]*h_fc_2[i+1])
            #print(sess.run(_asso_loss))
            asso_loss+=_asso_loss
    return asso_loss
smooth_loss=smooth_loss(h_fc_1)
prediction_loss=prediciton_loss(h_fc_1,Y)
association_loss=association_loss(h_fc_2)
print(sess.run(smooth_loss))
print(sess.run(prediction_loss))
print(sess.run(association_loss))
total_loss=smooth_loss+prediction_loss+association_loss
print(sess.run(total_loss))
train = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

#correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(y_input,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())
for i in range(100):
    cost, _ = sess.run([total_loss, train])
    print("train cost={:.6f}".format(cost))

